In [2]:
import numpy as np
np.random.seed(1234)

* = elementwise multiplication

outer = elementwise

dot = normal dot product

Simple Example

In [4]:
W = np.array(np.reshape([i for i in range(1,13)], (4,3)))
x = np.array([1,2,3])
u = np.array([-1,0,2,4])

def dot(W, x):
    value = np.dot(W, x)

    def vjp(u):
        vjp_wrt_W = np.outer(u, x)  #applied to W
        vjp_wrt_x = W.T.dot(u)  #applied to x
        # return vjp_wrt_W, vjp_wrt_x
        return vjp_wrt_x, vjp_wrt_W
        
    return value, vjp

value, vjp = dot(W, x)
print(value)
print(vjp(u))

[14 32 50 68]
(array([53, 58, 63]), array([[-1, -2, -3],
       [ 0,  0,  0],
       [ 2,  4,  6],
       [ 4,  8, 12]]))


### Question 1 
Implement the relu function and its VJP in the format above. Using the finite difference equation (slide 13), make sure that the VJP is correct numerically.

In [6]:
def relu(x):
    value = np.maximum(0, x)

    def vjp(u):
        gdash = lambda y: 1 if y>=0 else 0
        vjp_wrt_x = u*np.vectorize(gdash)(x)
        return vjp_wrt_x,  
        # The comma is important!
    
    return value, vjp


x = np.array([1,2,3])
u = np.array([-1,0,2])
value, vjp = relu(x)

In [36]:
def act_tanh(x):
    value = np.tanh(x)

    def vjp(u): 
        gdash = lambda z: (1/np.cosh(z))**2         #Is this even right?
        vjp_wrt_x = u*np.vectorize(gdash)(x)
        return vjp_wrt_x,  
    
    return value, vjp

x = np.array([1,2,3])
u = np.array([-1,0,2])
value, vjp = act_tanh(x)

In [31]:
def testFinDiff(ftrue, ftest, eps, d, W=[]):
    x = np.random.randn(d)
    val, vjp = ftest(x)

    g = lambda x, eps, e: ( (ftrue(x+eps*e) - ftrue(x)) / eps )

    fin_diff = []
    for i in range(0, len(x)):
        fin_diff.append(g(x, eps, np.eye(1,len(x),i).reshape(-1)))
    fin_diff = np.array(fin_diff).T

    #VJP with ei extracts ith row
    calculated_vjpX = []
    for i in range(len(x)):
        vjpxi, = vjp(np.eye(1,len(x),i).reshape(-1))
        calculated_vjpX.append(vjpxi)
    calculated_vjpX = np.array(calculated_vjpX)

    return (np.allclose(fin_diff, calculated_vjpX, rtol=1e-6))

assert(testFinDiff(lambda x: np.maximum(0, x), relu, 1e-8, 4))
print("Pass: Equal VJP's")

Pass: Equal VJP's


### **Question 2**
Reusing dot and relu, implement a 2-layer MLP with a relu activation.

The 2-layer MLP is defined as follows:
$$x\mapsto\text{relu}(W_{1}x)$$

In [11]:
def mlp1(x, W1):
    a, vjp1 = dot(W1, x)
    b, vjp2 = relu(a)
    
    value = b
    def vjp(u):
        vjp_wrt_a, = vjp2(u)
        # vjp_wrt_W1, vjp_wrt_x = vjp1(vjp_wrt_a)
        vjp_wrt_x, vjp_wrt_W1 = vjp1(vjp_wrt_a)
        return vjp_wrt_x, vjp_wrt_W1
    return value, vjp

#W1: First layer weights; has shape (D, H)

#Little test
D = 4
H = 3
W1 = np.random.rand(D,H)
x = np.random.rand(H)
u = np.random.rand(D)

val, vjp = mlp1(x, W1)
print(val)
print(vjp(u))


[1.34936185 0.86358828 1.41374657 1.27940146]
(array([1.37454734, 0.70825269, 0.71636386]), array([[0.12495168, 0.06270727, 0.11532222],
       [0.6120925 , 0.30717992, 0.56492128],
       [0.61237091, 0.30731964, 0.56517824],
       [0.19015821, 0.0954313 , 0.17550357]]))


In [38]:
def mlp2(x, W2, W1):
    """
    input: 
        x = input data
        W1 = weight matrix
        W2 = weight matrix
    formula:
        y = W2.q(W1.x)
    returns:
        value = evaluated value according to formula
        vjp = tuple of vjp's in order d/dx, d/dW1, d/dW2

    """
    a, vjp_dot1 = dot(W1, x)
    b, vjp_relu = relu(a)
    c, vjp_dot2 = dot(W2, b)
    value = c

    def vjp(u):
        # vjp_wrt_W2, vjp_wrt_b = vjp_dot2(u)
        vjp_wrt_b, vjp_wrt_W2 = vjp_dot2(u)
        vjp_wrt_a, = vjp_relu(vjp_wrt_b)
        # vjp_wrt_W1, vjp_wrt_x = vjp_dot1(vjp_wrt_a) 
        vjp_wrt_x, vjp_wrt_W1 = vjp_dot1(vjp_wrt_a) 

        return vjp_wrt_x, vjp_wrt_W1, vjp_wrt_W2 
    return value, vjp



def mlp3(x, W):
    value, vjp_1 = mlp2(x, W[-2], W[-1])
    value, vjp_2 = relu(value)
    value, vjp_3 = dot(W[-3], value)

    def vjp(u):
        vjp_wrt_Wk, vjp_wrt_x = vjp_3(u)    #order must actually be changed here
        vjp_wrt_x, = vjp_2(vjp_wrt_x)
        vjp_wrt_x_wrtW = vjp_1(vjp_wrt_x)
        return vjp_wrt_x_wrtW, vjp_wrt_Wk

    return value, vjp
    


In [39]:
D, H, C = [3,2,4]

x = np.random.rand(H)
W1 = np.random.rand(D,H)
W2 = np.random.rand(C,D)
# W3 = np.random.rand(C,C)
# u = np.random.rand(C)
W3 = np.random.rand(H,C)
W4 = np.random.rand(D,H)
W5 = np.random.rand(D,D)
u = np.random.rand(D)

# val, vjp = mlp2(x, W2, W1)
# val, vjp = mlp3(x, [W3, W2, W1])
# val, vjp = mlpk(x, [W5, W4, W3, W2, W1])

# print(val)
# print(vjp(u))


### **Question 3** 
Implement the squared loss VJP

In [40]:
def squared_loss(y_pred, y):
    residual = y_pred - y
    
    def vjp(u):
        vjp_y_pred = u*(1*residual)
        vjp_y = u*(-1*residual)
        return vjp_y_pred, vjp_y

    value = 0.5 * np.sum(residual ** 2)
    # The code requires every output to be an array.
    return np.array([value]), vjp

y = np.random.rand(5)
epsilon = np.random.uniform(-1, 1, 5)/5
y_pred = y + epsilon
u = np.array([1,2,3,4,5])

val, vjp = squared_loss(y_pred, y)

### **Question 4**
Implement the loss by composing mlp2 and squared_loss

In [41]:
def loss(x, y, W2, W1):
    # pred, predicted_vjp = mlp2(x, W1, W2) #Elysheva suggested a change as follows:
    pred_value, predicted_vjp = mlp2(x, W2, W1)
    loss_value, loss_vjp = squared_loss(pred_value, y)
    value = loss_value

    def vjp(u):
        vjp_y, vjp_y_pred = loss_vjp(u)
        vjp_x, vjp_W1, vjp_W2 = predicted_vjp(vjp_y_pred)
        return vjp_x, vjp_y, vjp_W1, vjp_W2
    
    return value, vjp

# y = np.random.rand(C)
# u = np.random.rand(C)

# val, vjp = loss(x, y, W1, W2)
# print(val)
# print(vjp(u))


### **Question 5** 
Implement an MLP with an arbitrary number of layers.

In [42]:
def initialiseMLP_random(inputfeatures, layers, verbose=False):
    import random
    dims = random.choices([i for i in range(2,11)], k=layers)
    W = [np.random.rand(dims[0], inputfeatures)]
    for i in range(1, len(dims)):
        Wi = np.random.rand(dims[i], dims[i-1])
        W.append(Wi)

    W.reverse()
    x = np.random.random(inputfeatures)
    u = np.random.rand(dims[-1])

    if verbose:
        print(np.shape(x))
        for i in W:
            print(np.shape(i))
        print(np.shape(u))

    return x, W, u

In [43]:
def mlpk(x, W):
    """
    input:
        x = input data
        W = list of weight matrices ordered Wk, ..., W2, W1
    formula:
        Wk.q(Wk-1q(...W2q(W1x)))
        mlp2(mlp2(mlp2(x, W2, W1), W3, W2), W4, W3)
    returns
        value = evaluated value of network
        vjp = tuple of vjp's in order d/dx, d/dW1, ... , d/dWk
    """
    if (len(W)>3):
        # print("if", len(W))
        value, vjp_1 = mlpk(x, W[1:len(W)])
    else:
        # print("else", len(W))
        value, vjp_1 = mlp2(x, W[-2], W[-1])
    
    value, vjp_2 = relu(value)
    value, vjp_3 = dot(W[0], value)

    def vjp(u):
        # vjp_wrt_Wk, vjp_wrt_x = vjp_3(u)
        vjp_wrt_x, vjp_wrt_Wk = vjp_3(u)
        vjp_wrt_x, = vjp_2(vjp_wrt_x)
        vjp_wrt_x_wrtW = vjp_1(vjp_wrt_x)
        return vjp_wrt_x_wrtW, vjp_wrt_Wk

    return value, vjp


In [44]:
x, W, u = initialiseMLP_random(4, 7, verbose=False)
val, vjp = mlpk(x, W)

print(val)
vjp_output = list(vjp(u))
# print("vjp_x", vjp_output[0], sep='\n')
print(len(vjp_output[0]))
i=0
# for w in vjp_output[1]:
    # print("W{i}".format(i=i), w, sep='\n')
    # i=i+1


[468.76638974 249.83885438 488.99532857 592.17834559]
2


Check implementation by checking gradient

In [45]:
import scipy
def check_grad_calculations(pblinreg, pblogreg, n, d):
    from scipy.optimize import check_grad 
    print(check_grad(pblinreg.fun, pblinreg.grad, np.random.randn(d)))
    grad_error = []
    for i in range(n):
        ind = np.random.choice(n,1)
        w =  np.random.randn(d)
        vec =  np.random.randn(d)
        eps = pow(10.0, -7.0)
        grad_error.append((pblinreg.f_i( ind[0], w+eps*vec) - pblinreg.f_i( ind[0], w))/eps - np.dot(pblinreg.grad_i(ind[0],w),vec)) 
    print(np.mean(grad_error))

    # Check for the logistic regression problem
    print(check_grad(pblogreg.fun, pblogreg.grad, np.random.randn(d)))
    grad_error = []
    for i in range(n):
        ind = np.random.choice(n,1)
        w =  np.random.randn(d)
        vec =  np.random.randn(d)
        eps = pow(10.0, -7.0)
        grad_error.append((pblogreg.f_i( ind[0], w+eps*vec) - pblogreg.f_i( ind[0], w))/eps - np.dot(pblogreg.grad_i(ind[0],w),vec)) 
    print(np.mean(grad_error))





In [46]:
def mlp2(x, W1, W2):
    a, vjp_a = mlp1(x, W1)
    b, vjp_b = mlp1(a, W2)

    def vjp(u):
        vjp_wrt_x, vjp_wrt_W2 = vjp_b(u)
        vjp_wrt_x, vjp_wrt_W1 = vjp_a(vjp_wrt_x)

        return vjp_wrt_x, vjp_wrt_W1, vjp_wrt_W2
    
    return b, vjp


    

D, H, C = [3,2,4]
x = np.random.rand(H)
W1 = np.random.rand(D,H)
W2 = np.random.rand(C,D)
u = np.random.rand(C)

val, vjp = mlp2(x, W1, W2)
print(val)
print(vjp(u))

[0.40252714 0.34232166 0.50288791 0.33283686]
(array([0.88500466, 1.25942425]), array([[0.85765147, 0.09564121],
       [0.74399662, 0.08296697],
       [0.71224606, 0.07942629]]), array([[0.2328898 , 0.13321321, 0.13256135],
       [0.17788106, 0.10174816, 0.10125026],
       [0.03340358, 0.01910688, 0.01901338],
       [0.08097938, 0.04632029, 0.04609363]]))


### **Question 6**  
Implement SGD to train your MLP on a dataset of your choice. Study the impact of depth (number of layers) and width (number of hidden units).

In [1]:
""" 
SGD: x_k+1 = x_k + a_k * del_f_i(x_k)
"""
from sklearn import datasets
dataset = datasets.fetch_california_housing(as_frame = True)

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import numpy as np
np.random.seed(1)

dataset.frame_normalized = StandardScaler().fit_transform(dataset.frame)
# We drop Longitude as well since Latitude has enough information
X = dataset.frame_normalized[:,0:len(dataset.frame.columns) - 2]
y = dataset.frame_normalized[:,len(dataset.frame.columns) - 1]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 9)
X_train = np.insert(X_train, 0, np.ones(X_train.shape[0]), axis=1)
X_test = np.insert(X_test, 0, np.ones(X_test.shape[0]), axis=1)

n, d = X_train.shape


In [51]:
def loss_i(i, X, y, W):
    pred_value, predicted_vjp = mlpk(X[i], W)
    loss_value, loss_vjp = squared_loss(pred_value, y)
    value = loss_value

    def vjp(u):
        vjp_y, vjp_y_pred = loss_vjp(u)
        vjp_x, vjp_W1, vjp_W2 = predicted_vjp(vjp_y_pred)
        return vjp_x, vjp_y, vjp_W1, vjp_W2
    
    return value, vjp

In [54]:
innerdim = 5
W0 = np.ones((d, innerdim))
W1 = np.ones((innerdim, innerdim))
W2 = np.ones((innerdim, 1))
W = [W0, W1, W1, W1, W2]

In [57]:
for i in W:
    print(i.shape)

(8, 5)
(5, 5)
(5, 5)
(5, 5)
(5, 1)


In [ ]:
def SGD(X, y, Wstar, niter, step):
    n, d = X.shape
    for it in range(niter):
        i = np.random.choice(n, 1)
        vali, vjpi = loss_i(i, X, y, W)
        ei = np.eye(d)[i]
        vjpi = vjpi(ei)
        # for k in range(len(W)):
        print(i, vjpi)

In [59]:
SGD(X_train, y_train, W, niter=2, step=0.05)

ValueError: shapes (5,5) and (1,8) not aligned: 5 (dim 1) != 1 (dim 0)